<a href="https://colab.research.google.com/github/SwathiAkiti/NeuralNetwork/blob/main/700748992_MiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Define data paths
data_dir = '/content/COVID-19_Radiography_Dataset'
batch_size = 32
img_size = (224, 224)

# Data Preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 33866 images belonging to 4 classes.
Found 8464 images belonging to 4 classes.


In [ ]:
# GDCNN Model Architecture
def build_gdcnn(input_shape=(224, 224, 3), num_classes=4): # Change num_classes to 4
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)

    x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    x = DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x) # This layer now outputs 4 classes

    model = Model(inputs, outputs)
    return model

In [ ]:
# Build and compile the model
model = build_gdcnn(input_shape=(224, 224, 3), num_classes=4) # Change num_classes to 4 to match the dataset
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Model Training
epochs = 1
history = model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    verbose=1
)

# Model Evaluation
loss, accuracy, precision, recall = model.evaluate(val_gen)
specificity = 0  # Placeholder for specificity calculation, to be added based on predictions

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Sensitivity (Recall): {recall*100:.2f}%")
print(f"Specificity: {specificity*100:.2f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   8/1059 ━━━━━━━━━━━━━━━━━━━━ 2:06:25 7s/step - accuracy: 0.3262 - loss: 1.3877 - precision: 0.3496 - recall: 0.0156